# Api URLS
## https://www.alphavantage.co/documentation/
## https://datahub.io/core/s-and-p-500-companies#data-cli

In [2]:
# import dependencies
import csv

# API Dependencies
import requests
import json
from pandas.io.json import json_normalize # deal with nested columns in api

# data science libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Config file keys.  We can import multiple keys variales in here
from config import av_key # api key for alpha vantage

In [3]:
# determine the top companies from datahub stock cli
data_url = "https://datahub.io/core/s-and-p-500-companies-financials/r/constituents-financials.csv"
datahub_df = pd.read_csv(data_url) # raw_df

# sector data
sector_df = datahub_df.groupby("Sector").sum() 
most_price_df = sector_df.nlargest(3, 'Price')
most_price_df

,Price,Price/Earnings,Dividend Yield,Earnings/Share,52 Week Low,52 Week High,Market Cap,EBITDA,Price/Sales,Price/Book
Sector,,,,,,,,,,
Consumer Discretionary,10418.90,2119.49,132.082638,409.84,12342.2399,8071.758553,3213562747315,2.941510e+11,191.164584,2440.66
Information Technology,8347.00,2602.59,85.994554,217.70,9714.5049,6432.399100,6727121800912,4.313397e+11,411.609970,761.15
Health Care,8083.46,1406.27,55.951842,264.77,9806.2706,6326.874430,3244359043367,2.231056e+11,294.461598,540.34


In [5]:
# whichever sectors have the highester market cap will 

market_cap_df = sector_df.nlargest(3, 'Market Cap')

market_cap_df

,Price,Price/Earnings,Dividend Yield,Earnings/Share,52 Week Low,52 Week High,Market Cap,EBITDA,Price/Sales,Price/Book
Sector,,,,,,,,,,
Information Technology,8347.00,2602.59,85.994554,217.70,9714.5049,6432.39910,6727121800912,4.313397e+11,411.609970,761.15
Financials,6055.81,1169.53,137.172367,301.71,6923.8857,4943.22380,3442649464852,3.597660e+10,228.356354,458.00
Health Care,8083.46,1406.27,55.951842,264.77,9806.2706,6326.87443,3244359043367,2.231056e+11,294.461598,540.34


In [30]:
# ALPHA VANTAGE API SCRAPING

# will need function, symbol, and api key in order to import the json from the api
url_alpha_vantage = "https://www.alphavantage.co/" 

# url variables
function = "TIME_SERIES_DAILY"
symbol = "MSFT"
outputsize = "full" # full or compact (100 latest vs all data)
query = url_alpha_vantage + "query?" + "function=" + function + "&symbol=" + symbol +"&outputsize=" + outputsize + "&apikey=" + av_key  
url_response = requests.get(query).json()
print(query)
# pretty print the json
print(json.dumps(url_response, indent=4, sort_keys=True))

https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=MSFT&outputsize=full&apikey=A2JKUWFZLQ7633GK
{
    "Meta Data": {
        "1. Information": "Daily Prices (open, high, low, close) and Volumes",
        "2. Symbol": "MSFT",
        "3. Last Refreshed": "2019-03-01",
        "4. Output Size": "Full size",
        "5. Time Zone": "US/Eastern"
    },
    "Time Series (Daily)": {
        "1998-01-02": {
            "1. open": "129.6300",
            "2. high": "131.5000",
            "3. low": "129.5000",
            "4. close": "131.1300",
            "5. volume": "4968500"
        },
        "1998-01-05": {
            "1. open": "131.2500",
            "2. high": "133.6300",
            "3. low": "127.8700",
            "4. close": "130.3800",
            "5. volume": "10047200"
        },
        "1998-01-06": {
            "1. open": "129.7500",
            "2. high": "133.0000",
            "3. low": "129.2500",
            "4. close": "131.1300",
            "5. vo

In [33]:
# create a data frame from the json
timeseries_df = pd.DataFrame.from_dict(url_response["Time Series (Daily)"])
microsoft_df = pd.DataFrame.from_dict(url_response, orient = 'columns')

# only look at data from the past 3 years 
#microsoft_df = microsoft_df.tail(730)
timeseries_df = timeseries_df.T.head(730)

display(timeseries.head(730))

,1. open,2. high,3. low,4. close,5. volume
2019-03-01,112.8900,113.0200,111.6650,112.5300,23501169
2019-02-28,112.0400,112.8800,111.7300,112.0300,29083934
2019-02-27,111.6900,112.3600,110.8800,112.1700,21487062
2019-02-26,111.2600,113.2400,111.1700,112.3600,21536733
2019-02-25,111.7600,112.1800,111.2600,111.5900,23750599
2019-02-22,110.0500,111.2000,109.8200,110.9700,27763218
2019-02-21,106.9000,109.4800,106.8700,109.4100,29063231
2019-02-20,107.8600,107.9400,106.2900,107.1500,21607671
2019-02-19,107.7900,108.6600,107.7800,108.1700,18038460
2019-02-15,107.9100,108.3000,107.3624,108.2200,26606886
